In [ ]:
from seqdata.core import *
from seqdata.model import *
from fastai2.basics import *
from fastai2.callback.progress import *

In [ ]:
#hide
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [ ]:
hdf_files = get_hdf_files('test_data/')
src_df = CreateDict([DfHDFCreateWindows(win_sz=1000+1,stp_sz=30,clm='current')])(hdf_files)
src_df

[{'path': Path('test_data/train/Sim_RealisticCycle2.hdf5'),
  'l_slc': 0,
  'r_slc': 1001},
 {'path': Path('test_data/train/Sim_RealisticCycle2.hdf5'),
  'l_slc': 30,
  'r_slc': 1031},
 {'path': Path('test_data/train/Sim_RealisticCycle2.hdf5'),
  'l_slc': 60,
  'r_slc': 1061},
 {'path': Path('test_data/train/Sim_RealisticCycle2.hdf5'),
  'l_slc': 90,
  'r_slc': 1091},
 {'path': Path('test_data/train/Sim_RealisticCycle2.hdf5'),
  'l_slc': 120,
  'r_slc': 1121},
 {'path': Path('test_data/train/Sim_RealisticCycle2.hdf5'),
  'l_slc': 150,
  'r_slc': 1151},
 {'path': Path('test_data/train/Sim_RealisticCycle2.hdf5'),
  'l_slc': 180,
  'r_slc': 1181},
 {'path': Path('test_data/train/Sim_RealisticCycle2.hdf5'),
  'l_slc': 210,
  'r_slc': 1211},
 {'path': Path('test_data/train/Sim_RealisticCycle2.hdf5'),
  'l_slc': 240,
  'r_slc': 1241},
 {'path': Path('test_data/train/Sim_RealisticCycle2.hdf5'),
  'l_slc': 270,
  'r_slc': 1271},
 {'path': Path('test_data/train/Sim_RealisticCycle2.hdf5'),
  'l_

In [ ]:
tfms=[  [HDF2Sequence(['current','voltage']),SeqSlice(l_slc=1),toTensorSequencesInput],
        [HDF2Sequence(['voltage']),SeqSlice(r_slc=-1),toTensorSequencesOutput]]
dsrc = DataSource(src_df,tfms=tfms,splits=ApplyToDict(ParentSplitter())(src_df))

In [ ]:
# db = dsrc.databunch(bs=128,after_batch=[SeqNoiseInjection(std=[1.1,0.01]),Normalize(axes=[0,1])])
db = dsrc.databunch(bs=128,after_batch=[Cuda(),Normalize(axes=[0,1])])
db.one_batch()[0].shape

torch.Size([128, 1000, 2])

In [ ]:
lrn = Learner(db,SimpleGRU(2,1),loss_func=nn.MSELoss())

In [ ]:
%%prun -s cumulative -l 20 -D batch.prof
lrn.fit(1)

epoch,train_loss,valid_loss,time
0,0.148762,0.008298,00:16


 
*** Profile stats marshalled to file 'batch.prof'. 


In [ ]:
%lprun?

In [ ]:
%%prun -s cumulative -l 20 -D batch.prof
db.one_batch()

 
*** Profile stats marshalled to file 'batch.prof'. 


In [ ]:
%%time
%lprun -f dsrc.tfms[1][0].init_enc -f dsrc.tls.__getitem__ db.one_batch()

CPU times: user 677 ms, sys: 5.59 ms, total: 682 ms
Wall time: 680 ms


In [ ]:
%lprun -f DataLoader.one_batch db.one_batch()

In [ ]:
tfms=[  [HDF2Sequence(['current','voltage'])],
        [HDF2Sequence(['voltage']),SeqSlice(r_slc=-1),toTensorSequencesOutput]]
dsrc = DataSource(src_df,tfms=tfms,splits=ApplyToDict(ParentSplitter())(src_df))

In [ ]:
%%timeit
dsrc[0]

403 µs ± 2.17 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
#hide
from nbdev.export import *
notebook2script()